In [10]:
from sympy import *
from sympy.integrals.manualintegrate import integral_steps
import numpy as np
from itertools import product, combinations_with_replacement, chain
from operator import itemgetter, add
from math import factorial
from multiprocessing import Pool, cpu_count
import re
import json
import math
from sympy.physics.units.quantities import Quantity
k = Quantity('k')
x,a,b,c = symbols('x a b c', real=True)
# 2*(1026!/(1026-2)!)+((1026+1)!/(2*(1026-1)!))+(1026!/(2*(1026-2)!))+4*1026

In [96]:
f = (x + 3)/cos(x)
domain = calculus.util.continuous_domain(f, x, S.Reals)
all_zeros = solveset(f, x, domain)
zeros_count = oo
print(type(all_zeros))
print(srepr(all_zeros))
curr_args = all_zeros
while isinstance(curr_args, Intersection) or isinstance(curr_args, Complement):
    if len(curr_args.args) == 0:
        break
    first_arg = curr_args.args[0]
    if first_arg.is_FiniteSet:
        print(first_arg)
        break
    curr_args = first_arg
if isinstance(all_zeros, Intersection):
    finite_sets = [arg for arg in all_zeros.args if arg.is_FiniteSet]
    if len(finite_sets) > 0:
        all_included = all([f.subs(x, z) == 0 for z in finite_sets[0]])
        if all_included:
            zeros_count = len(finite_sets[0])
print(zeros_count)
all_zeros

<class 'sympy.sets.sets.Complement'>
Complement(Intersection(FiniteSet(Integer(-3)), Complement(Reals, Union(ImageSet(Lambda(Tuple(Dummy('n', dummy_index=2364638)), Add(Mul(Integer(2), Dummy('n', dummy_index=2364638), pi), Mul(Rational(1, 2), pi))), Integers), ImageSet(Lambda(Tuple(Dummy('n', dummy_index=2364643)), Add(Mul(Integer(2), Dummy('n', dummy_index=2364643), pi), Mul(Rational(3, 2), pi))), Integers)))), Complement(ImageSet(Lambda(Tuple(Dummy('n', dummy_index=2364756)), Add(Mul(Integer(2), Dummy('n', dummy_index=2364756), pi), Mul(Rational(3, 2), pi))), Integers), Union(ImageSet(Lambda(Tuple(Dummy('n', dummy_index=2364638)), Add(Mul(Integer(2), Dummy('n', dummy_index=2364638), pi), Mul(Rational(1, 2), pi))), Integers), ImageSet(Lambda(Tuple(Dummy('n', dummy_index=2364643)), Add(Mul(Integer(2), Dummy('n', dummy_index=2364643), pi), Mul(Rational(3, 2), pi))), Integers))))
{-3}
oo


Complement(Intersection(Complement(Reals, Union(ImageSet(Lambda(_n, 2*_n*pi + pi/2), Integers), ImageSet(Lambda(_n, 2*_n*pi + 3*pi/2), Integers))), {-3}), Complement(ImageSet(Lambda(_n, 2*_n*pi + 3*pi/2), Integers), Union(ImageSet(Lambda(_n, 2*_n*pi + pi/2), Integers), ImageSet(Lambda(_n, 2*_n*pi + 3*pi/2), Integers))))

In [13]:
from sympy.solvers.solveset import solveset


def get_singularities(expression, symbol, domain=None):
    if domain is None:
        domain = S.Reals if symbol.is_real else S.Complexes
    try:
        sings = S.EmptySet
        for i in expression.rewrite([sec, csc, cot, tan], cos).atoms(Pow):
            print(i)
            if i.exp.is_infinite:
                raise NotImplementedError
            if i.exp.is_negative:
                print('durch 0 teilen', i.base)
                print(solve(i.base, x))
                sings += solveset(i.base, symbol, domain)
        for i in expression.atoms(log):
            print('logarithmus <= 0')
            sings += solveset(i.args[0], symbol, domain)
        return sings
    except NotImplementedError:
        raise NotImplementedError(filldedent('''
            Methods for determining the singularities
            of this function have not been developed.'''))
    
get_singularities(2/(x+exp(x)), x)

1/(x*exp(x) + x)
durch 0 teilen x*exp(x) + x
[0]


{0}

In [11]:
f = sin(x)
fd = diff(f,x)
fdd = diff(fd,x)
fddd = diff(fdd,x)
s_set = solveset(fdd, x, Interval(0, 2*pi))
# solveset(fdd > 0, x, s_set)
s_set

{0, pi, 2*pi}

In [12]:
x = Symbol('x', real=True)
y = Symbol('y', real=True)
f = exp(x)-exp(-x)
inverses = [i.subs(y, x) for i in solve(f-y, x) if not i.has(I)]
inverses

[log(x/2 - sqrt(x**2 + 4)/2), log(x/2 + sqrt(x**2 + 4)/2)]

In [31]:
a = Symbol('a', positive=True)
b = Symbol('b', negative=True)
f = (a+x)/(b+x)
zeros = solveset(f, x, S.Reals)
len(zeros)

1

In [47]:
f = log(x)*cos(x)
step = integral_steps(f, x)
for s in preorder_traversal(step):
    print(s)
rules = set()

def add_rules(step):
    if hasattr(step, 'v_step'):
        print(type(step.v_step).__name__)
        print(step.v_step)
        rules.add(type(step.v_step).__name__)
    if hasattr(step, 'second_step'):
        type(step.second_step).__name__
        rules.add(type(step.second_step).__name__)
    rules.add(type(step).__name__)

while hasattr(step, 'substep') or hasattr(step, 'alternatives'):
    if hasattr(step, 'alternatives'):
        step = step.alternatives[0]
    else:
        add_rules(step)
        step = step.substep
add_rules(step)
rules

AlternativeRule(alternatives=[URule(u_var=_u, u_func=log(x), constant=1, substep=PartsRule(u=_u, dv=exp(_u)*cos(exp(_u)), v_step=URule(u_var=_u, u_func=exp(_u), constant=1, substep=TrigRule(func='cos', arg=_u, context=cos(_u), symbol=_u), context=exp(_u)*cos(exp(_u)), symbol=_u), second_step=URule(u_var=_u, u_func=exp(_u), constant=1, substep=SiRule(a=1, b=0, context=sin(_u)/_u, symbol=_u), context=sin(exp(_u)), symbol=_u), context=_u*exp(_u)*cos(exp(_u)), symbol=_u), context=log(x)*cos(x), symbol=x), PartsRule(u=log(x), dv=cos(x), v_step=TrigRule(func='cos', arg=x, context=cos(x), symbol=x), second_step=SiRule(a=1, b=0, context=sin(x)/x, symbol=x), context=log(x)*cos(x), symbol=x)], context=log(x)*cos(x), symbol=x)
[URule(u_var=_u, u_func=log(x), constant=1, substep=PartsRule(u=_u, dv=exp(_u)*cos(exp(_u)), v_step=URule(u_var=_u, u_func=exp(_u), constant=1, substep=TrigRule(func='cos', arg=_u, context=cos(_u), symbol=_u), context=exp(_u)*cos(exp(_u)), symbol=_u), second_step=URule(u_va

{'PartsRule', 'URule'}

In [64]:
valid_attrs = ['substep', 'substeps', 'v_step', 'second_step', 'alternatives']
ignore_rules = ['AlternativeRule']

def get_rules(step):
    rule = type(step).__name__
    if rule not in ignore_rules:
        yield rule
    for attr in valid_attrs:
        if hasattr(step, attr):
            if attr == 'alternatives':
                yield from get_rules(step.alternatives[0])
            elif attr == 'substeps':
                for substep in step.substeps:
                    yield from get_rules(substep)
            else:
                yield from get_rules(getattr(step, attr))

f = (x + 4)**2
step = integral_steps(f, x)
print(step)
rules = list(dict.fromkeys((get_rules(step))))
rules

AlternativeRule(alternatives=[URule(u_var=_u, u_func=x + 4, constant=1, substep=PowerRule(base=_u, exp=2, context=_u**2, symbol=_u), context=(x + 4)**2, symbol=x), RewriteRule(rewritten=x**2 + 8*x + 16, substep=AddRule(substeps=[PowerRule(base=x, exp=2, context=x**2, symbol=x), ConstantTimesRule(constant=8, other=x, substep=PowerRule(base=x, exp=1, context=x, symbol=x), context=8*x, symbol=x), ConstantRule(constant=16, context=16, symbol=x)], context=x**2 + 8*x + 16, symbol=x), context=(x + 4)**2, symbol=x)], context=(x + 4)**2, symbol=x)


['URule', 'PowerRule']

In [99]:
f = -c*x**3 - c + x**2 + x
type(integrate(f, x, risch=True))

sympy.core.add.Add

In [2]:
f = -a/x+x+b
integral_steps(f, x)

AddRule(substeps=[ConstantTimesRule(constant=-a, other=1/x, substep=ReciprocalRule(func=x, context=1/x, symbol=x), context=-a/x, symbol=x), ConstantRule(constant=b, context=b, symbol=x), PowerRule(base=x, exp=1, context=x, symbol=x)], context=-a/x + b + x, symbol=x)

In [126]:
def depth(expr):
    print(expr.args, expr.func)
    if isinstance(expr, Atom):
        return 0
    else:
        inc = 1
        arg_types = [type(a) for a in expr.args]
        if expr.func == Mul and -1 in expr.args:
            print('mul -1')
            inc = 0
        elif expr.func == Mul and arg_types == [Integer, Pow] and -1 in expr.args[1].args:
            print('pow -1')
            inc = 0
        return inc + max([depth(arg) for arg in expr.args])
    
f = 2/(1-x)
depth(f)


(2, 1/(1 - x)) <class 'sympy.core.mul.Mul'>
pow -1
() <class 'sympy.core.numbers.Integer'>
(1 - x, -1) <class 'sympy.core.power.Pow'>
(1, -x) <class 'sympy.core.add.Add'>
() <class 'sympy.core.numbers.One'>
(-1, x) <class 'sympy.core.mul.Mul'>
mul -1
() <class 'sympy.core.numbers.NegativeOne'>
() <class 'sympy.core.symbol.Symbol'>
() <class 'sympy.core.numbers.NegativeOne'>


2

In [9]:
f = x*exp(x)/4
# f = (2+x)*(x+3)

class ArgStack:
    def __init__(self):
      self.stack = []

    def __len__(self):
        return len([a for a, skip in self.stack if not skip])
  
    def push(self, n_args, skip):
        self.stack.append([n_args, skip])
  
    def pop(self):
        if not len(self.stack):
            return
        if self.stack[-1][0] <= 1:
            slice_end = 0
            for n, _ in reversed(self.stack):
                if n > 1:
                    break
                else:
                    slice_end -= 1
            self.stack = self.stack[:slice_end]
        if len(self.stack):
            self.stack[-1][0] -= 1

# this function is needed, because sympy always converts subtraction to addition
# i.e. 1-x to 1+(-1)*x which introduces an extra leaf, node and level of depth
# it also always converts division to multiplication
# i.e. 4/(1-x) to 4*(1-x)^-1 which also introduces an extra leaf, node and level of depth
def tree_props(f):
    leaves = 0
    nodes = 0
    arg_stack = ArgStack()
    max_depth = 0
    marked_pows = []
    for expr in preorder_traversal(f):
        skip_depth = False
        if expr.func == Mul:
            arg_types = set([type(a) for a in expr.args])
            # multiplication with -1
            if -1 in expr.args:
                leaves -= 1
                nodes -= 2 # nodes are always decreased by two, because *(-1) has two nodes
                skip_depth = True
            # power of -1 which replaces division (can have multiple in one multiplication)
            if Pow in arg_types:
                # have to mark these pows for later, because you can't retrieve parents of args
                pows = [arg for arg in expr.args if type(arg) == Pow and -1 in arg.args]
                marked_pows += pows
                # a multiplication of -1 and a marked pow in the same expression would reduce leaves and depth too much
                if len(pows) and -1 in expr.args:
                    leaves += 1
                    nodes += 2
                    skip_depth = False

        if expr.func == Pow and expr in marked_pows:
            leaves -= 1
            nodes -= 2
            skip_depth = True
            marked_pows.remove(expr)
        
        if isinstance(expr, Atom):
            leaves += 1
            arg_stack.pop()
        else:
            arg_stack.push(len(expr.args), skip_depth)
        nodes += max(1, len(expr.args) - 1)
        max_depth = max(max_depth, len(arg_stack))
    return (max_depth, leaves, nodes)

In [16]:
assert tree_props(x**2) == (1,2,3)
assert tree_props(1/x) == (1,2,3)
assert tree_props(2/x) == (1,2,3)
assert tree_props(x+1) == (1,2,3)
assert tree_props(x-1) == (1,2,3)
assert tree_props(x-2) == (1,2,3)
assert tree_props(1-x) == (1,2,3)
assert tree_props(2-x) == (1,2,3)
assert tree_props(x/3) == (1,2,3)
assert tree_props((2+x)*(x+3)) == (2,4,7)
assert tree_props(x/(2*x + 8)) == (3,4,7)
assert tree_props(x**2/(x - 2)) == (2,4,7)
assert tree_props(exp(sin(x))) == (2,1,3)
assert tree_props(exp(sin(x))+exp(cos(x))) == (3,2,7)
assert tree_props(-2-x + acos(x)) == (2,3,6)
assert tree_props(-x**2 + log(x)) == (2,3,6)
assert tree_props(4*x/(x + 4)) == (2,4,7)
assert tree_props(2*x-1/(x+2)) == (3,5,9)
assert tree_props(1/(x-3)-1/(x+2)) == (3,6,11)
assert tree_props(4/(x-3)-5/(x+2)) == (3,6,11)
assert tree_props(x*exp(x)/4) == (2,3,6) # multiplication with 3 operands
assert tree_props(sqrt(1 - x**2)) == (3,4,7) # sqrt = ^1/2, so it counts as a leaf & node too
assert tree_props(1/(8-x)+exp(7/(sin(x)+5))) == (5,6,13)

(4/(x-3)-5/(x+2)).count_ops(visual=True)

ADD + 2*DIV + 2*SUB

In [36]:
with open('uniques_ext_depth2.csv', 'r') as readfile:
    with open('tagged0.csv', 'w') as writefile:
        writefile.write('fn,depth\n')
        for line in readfile:
            writefile.write(line.strip() + ',')
            writefile.write(str(depth(sympify(line.strip()))) + '\n')

In [101]:
# Calculation method taken from https://en.wikipedia.org/wiki/Asymptote#Elementary_methods_for_identifying_asymptotes
# Also mentioned in https://encyclopediaofmath.org/index.php?title=Asymptote
def asymptotes(f):
    a = set()
    for lim in [oo, -oo]:
        m = limit(f/x, x, lim)
        if m.is_real:
            n = limit(f-m*x, x, lim)
            if n.is_real:
                a.add(m*x+n)
    return a

In [102]:
f = (x + 1) / (1 + exp(-x))
asymptotes(f)

{0, x + 1}

In [103]:
s = singularities(1/x, x)
extrema = [(1, 'max')]
f = FiniteSet(sqrt(6), -sqrt(6))
l = list(s.union(f).union(FiniteSet(oo)))
l.sort()
l

[-sqrt(6), 0, sqrt(6), oo]

In [104]:
parse_expr('Union(Interval.open(-oo, 0), Interval.open(0, oo))')

Union(Interval.open(-oo, 0), Interval.open(0, oo))

In [19]:
from sympy.integrals.manualintegrate import manualintegrate, integral_steps
f = 2*x
# calculus.util.continuous_domain(f, x, S.Reals)
steps = integral_steps(f, x)
rules = set()
for match in re.finditer('\w+Rule', str(steps)):
    rules.add(match.group())
print(rules)

{'ConstantTimesRule', 'PowerRule'}


In [7]:
f = sympify('log(x)', locals={'x': x})
singularities(f, x)

{0}

In [58]:
f = log(x)
f.is_rational_function()

In [10]:
def test(t=5):
    print(t)
s = None
test(t=s or 10)

10


In [7]:
from sympy.integrals.manualintegrate import manualintegrate, integral_steps
f = log(x+2)
integral_steps(f, x)

AlternativeRule(alternatives=[URule(u_var=_u, u_func=x + 2, constant=1, substep=PartsRule(u=log(_u), dv=1, v_step=ConstantRule(constant=1, context=1, symbol=_u), second_step=ConstantRule(constant=1, context=1, symbol=_u), context=log(_u), symbol=_u), context=log(x + 2), symbol=x), PartsRule(u=log(x + 2), dv=1, v_step=ConstantRule(constant=1, context=1, symbol=x), second_step=RewriteRule(rewritten=1 - 2/(x + 2), substep=AddRule(substeps=[ConstantRule(constant=1, context=1, symbol=x), ConstantTimesRule(constant=-2, other=1/(x + 2), substep=URule(u_var=_u, u_func=x + 2, constant=1, substep=ReciprocalRule(func=_u, context=1/_u, symbol=_u), context=1/(x + 2), symbol=x), context=-2/(x + 2), symbol=x)], context=1 - 2/(x + 2), symbol=x), context=x/(x + 2), symbol=x), context=log(x + 2), symbol=x)], context=log(x + 2), symbol=x)

In [2]:
s = ''
if s:
    print('lol')

In [29]:
from sympy.integrals.risch import NonElementaryIntegral

f = 1/log(x)
i = integrate(f, x, risch=True)
len(i.atoms(NonElementaryIntegral))

1

In [7]:
lines = [
    '2*c + x',
    'c + x**2',
    'c + cos(x)',
    '2*c - x',
    'c + exp(x)',
    'c + x**2 + x',
    'c + c/x + x',
    'c + 2*x',
    'c + x + log(x)',
    'c + 2*x',
    '2*c + x',
]
re_mul_int = re.compile('(?<!\*)\*?\d\*(x|c)')

def replace_mul_int(match):
    if match.group(1) == 'x':
        return 'c*x'
    return 'c'

for line in lines:
    print(line)
    line = re.sub(
        re_mul_int,
        replace_mul_int,
        line
    )
    print(line)

2*c + x
c + x
c + x**2
c + x**2
c + cos(x)
c + cos(x)
2*c - x
c - x
c + exp(x)
c + exp(x)
c + x**2 + x
c + x**2 + x
c + c/x + x
c + c/x + x
c + 2*x
c + c*x
c + x + log(x)
c + x + log(x)
c + 2*x
c + c*x
2*c + x
c + x
